Ovo je nastavak price o sekvencioniranju genoma. Tada smo govorili o sekvencioniranju genoma na nivou vrste organizama, a sada govorimo o sekvencioniranju individualnih genoma u cilju lociranja mutacija koje izazivaju bolesti. 

Genom koji uzimamo kao reprezentativni genom jedne vrste organizama naziva se **referentni genom**. Kada smo sekvencirali referentni genom kretali smo sa sklapanjem od nule. Sada zelimo da iskoristimo to sto nam je poznat referentni genom i da na efikasniji nacin sastavljamo individualne genome koristeci referentni. To radimo **mapiranjem ocitavanja** individualnog genoma u referentnom genomu, tj. odredjivanjem pozicije u referentnom genomu sa kojima ocitavanja individualnog genoma imaju veliku slicnost. Na ovaj nacin smo sveli problem na problem **uparaivanja sablona** - da li je jedna niska (sablon) podniska druge niske.

# Prefiksna stabla

**Prefiksno stablo** (jos se naziva **trie**, od engleske reci reTRIEval) je struktura podataka u obliku uredjenog stabla koje se koristi za cuvanje vise niski (sablona) na takav nacin da se zajednicki prefiksi razlicitih niski predstavljaju istim putanjama od korena do tacke (cvora) razlikovanja. Osnovna ideja ovakve strukture podataka je da se omoguci istovremeno pretrazivanje vise niski (sablona) unutar druge niske (sekvence) na efikasan nacin. Svakoj od polaznih niski odgovara tacno po jedna putanja od korena do lista. Granama prefiksnog stabla su pridruzeni pojedinacni karakteri niski, dok se cvorovima pridruzuju (pod)niske koje se dobijaju nadovezivanjem karaktera koji se nalaze na granama duz putanje od korena do tog cvora (korenu odgovara prazna niska). Postoji vise razlicitih implementacija ovako definisane strukture podataka, u smislu toga sta se od informacija cuva u cvorovima a sta na granama stabla. 

<img src="assets/prefix_tree.png" width="450"> 

Uparivanje, tj. pretrazivanje sablona (pojedinacnih ocitavanja) iz prefiksnog stabla unutar druge sekvence (referentnog genoma) vrsimo tako sto krenemo redom po simbolima sekvence u kojoj trazimo sablone i pokusavamo da ih uparimo sa simbolima u stablu, pocev od korena, i ako na taj nacin stignemo do lista, onda zakljucujemo da se sablon koji odgovara putanji od korena do tog lista nalazi u datoj sekvenci. 

Poboljsanje prethodnog algoritma je u tome da se prolazak kroz nisku u kojoj trazimo sablone ne vrsi za po jedan pomeraj, vec potencijalno za vise (kao kod KMP algoritma). To cemo omoguciti dodavanjem dodatnih grana u stablu, tzv. **failure links**. Situacija u kojoj bi bilo pozeljno da se pomerimo za vise simbola odjendom jeste kada imamo da se **sufiks prethodnog sablona koji smo trazili (i nismo uspeli da ga nadjemo) preklapa sa prefiksom nekog drugog sablona**. Tada, ukoliko smo uparili prefiks prvog sablona i nismo uspeli, ne zelimo da se vracamo u koren i od pocetka uparujemo drugi sablon, vec da onaj deo koji se preklapa smatramo vec uparenim i da u niski u kojoj trazimo sablone nastavimo uparivanje tamo gde smo se zaustavili prilikom uparivanja prvog sablona. To omogucavamo failure linkom koji iz cvora u kojem smo naisli na nepoklapanje prilikom uparivanja prvog sablona vodi do cvora koji odgovara uparenju prefiksa drugog sablona. Ukoliko ne postoji drugi sablon ciji se prefiks poklapa sa sufiksom prethodnog sablona koji smo trazili, failure link vodi u koren stabla. **Ovaj algoritam je uopstenje KMP algoritma za visestruko uparivanje sablona.** 

Ovde cemo implementirati samo osnovnu varijantu prefiksnih stabala (bez poboljsanja sa failure link-ovima) i algoritma za pretragu sablona koriscenjem prefiksnog stabla. Nije nam cilj da postignemo maksimalnu efikasnost implementacije, vec samo da vidimo konceptualno princip primene prefiksnih stabala za ovakvu vrstu problema.

**NAPOMENA:** Pretpostavljacemo da nijedan sablon nije prefiks nekog drugog sablona (iako postoji nacin da se prevazidje ovo ogranicenje), tako da algoritam pronalazi uparivanje sa jednim sablonom, ili ni sa jednim.

Klasa **TrieNode** definise pojedinacne cvorove prefiksnog stabla.

In [ ]:
class TrieNode:
    def __init__(self, label):
        self.label = label
        self.neighbors = {}
        self.is_leaf = True

    def add_neighbor(self, character):
        self.is_leaf = False
        self.neighbors[character] = TrieNode(self.label + character)

    def has_neighbor(self, character):
        if character in self.neighbors:
            return True
        else:
            return False

    def get_neighbor(self, character):
        if character in self.neighbors:
            return self.neighbors[character]
        else:
            return None



In [1]:
class TrieNode:
    def __init__(self, label):                    
        self.label = label          #niska od nadovezanih karaktera sa grana na putu od korena do tog cvora
        self.neighbors = {}         #mapa suseda u formatu {karakter_pridruzen_grani : cvor_sused}
        self.is_leaf = True         #indikator da se u tom cvoru zavrsava jedna kompletna niska (sablon)
        
    def add_neighbor(self, character):
        self.is_leaf = False
        self.neighbors[character] = TrieNode(self.label + character)
        
    def has_neighbor(self, character):
        if character in self.neighbors:
            return True
        else:
            return False
        
    def get_neighbor(self, character):
        if self.has_neighbor(character):
            return self.neighbors[character]
        else:
            return None
        
    def get_neighbors(self):
        return self.neighbors

Klasa **Trie** definise prefiksno stablo kao i algoritam pretrage sekvence nad njime.

In [2]:
class Trie:
    #konstruktorska funkcija (metod) koja za datu
    #listu niski patterns pravi prefiksno stablo
    def __init__(self, patterns):
        self.root = TrieNode('')
    
        for pattern in patterns:
            current_node = self.root          #svaki pattern pocinje iz korena
        
            for current_symbol in pattern:
                if not current_node.has_neighbor(current_symbol):
                    current_node.add_neighbor(current_symbol)
                current_node = current_node.get_neighbor(current_symbol)
    
    #metod koji pokusava da upari sablone iz prefiksnog stabla kao prefikse niske sequence
    def prefix_trie_matching(self, sequence):
        current_node = self.root
    
        for character in sequence:
            if current_node.is_leaf:
                return current_node.label
            
            if current_node.has_neighbor(character):
                current_node = current_node.get_neighbor(character)
            else:
                return False
        
        #ovo je potrebno kada je sablon cela niska sequence
        if current_node.is_leaf:
            return current_node.label
        else:
            return False
        
    #metod koji pronalazi sva uparivanja sablona iz prefiksnog stabla unutar niske sequence    
    def trie_matching(self, sequence):
        results = []
    
        for i in range(len(sequence)):
            match = self.prefix_trie_matching(sequence[i:])
            if match:
                results.append((match, i))
            
        return results

In [3]:
patterns = ["ananas", "and", "antenna", "banana", "bandana", "nab", "nana", "pan"]

trie = Trie(patterns)

In [4]:
trie.prefix_trie_matching('panamabananas')

'pan'

In [5]:
trie.trie_matching('panamabananas')

[('pan', 0), ('banana', 6), ('ananas', 7), ('nana', 8)]

# Sufiksna stabla

**Sufiksno stablo** je struktura podataka koja pomocu uredjenog stabla opisuje internu strukturu (jedne) niske tako sto cuva sve sufikse date niske na takav nacin da svakom sufiksu odgovara po jedna putanja od korena do lista. Da bismo obezbedili da ce putanja pridruzena svakom sufiksu da se zavrsava u listu a ne u nekom unutrasnjem cvor, dodajemo na kraj svakog sufiksa specijalni terminirajuci karakter \$.

<img src="assets/suffix_tree.png" width="650"> 

Osnovna ideja ovakve strukture podataka je da se omoguci istovremeno pretrazivanje vise niski (sablona) unutar druge niske (sekvence) na efikasan nacin, sto je takodje bio slucaj kod prefiksnog stabla, s tim sto se sada stablo konstruise za nisku unutar koje se vrsi pretaga sablona. Dakle, pretpostavka je da ce se pretraga vrsiti uvek unutar jedne iste niske. Najcesca primena sufiksnih stabala jeste prebrojavanje broja pojavljivanja neke niske (sablona) unutar date niske (sekvence) za koju je konstruisano sufiksno stablo.

Za predstavljanje sufiksnog stabla mozemo iskoristiti prethodno definisanu klasu <code>Trie</code>.

# Sufiksni niz

**Sufiksni niz je samo kompaktnija reprezentacija sufiksnog stabla.** Nad sufiksnim nizom mozemo da vrsimo sve operacije kao nad sufiksnim stablom. To je niz koji sadrzi sve sufikse date niske (sa specijalnim terminirajucim karakterom) zajedno sa pocetnim pozicijama (indeksima) na kojima pocinje taj sufiks, i to sortirane leksikografski.

<img src="assets/suffix_array.png" width="350"> 

Uparivanje sablona vrsi se binarnom pretragom kroz sufiksni niz formiran za nisku koja se pretrazuje, tako sto se krene od sufiksa kojem odgovara srednja pozicija u sufiksnom nizu (koji je sortiran leksikografski) i uporedimo (leksikografski) sablon sa <u>prefiksom</u> tog sufiksa koji je jednake duzine kao sablon. Na osnovu leksikografskog poredjenja sablona i prefiksa srednjeg sufiksa zakljucujemo da li smo pronasli uparivanje ili odredjujemo u kojoj polovini sufiksnog niza treba nastaviti dalje pretragu. Pri tom, kada pronadjemo jedno uparivanje, potrebno je da proverimo i oko te pozicije u sufiksnom nizu da li mozemo da uparimo sablon sa sufiksima koji su leksikografski susedni pronadjenom uparivanju. 

Funkcija **generate_suffix_array** pravi sufiksni niz za nisku **sequence**.

In [7]:
def generate_suffix_array(sequence):
    suffixes = [(sequence[i:], i) for i in range(len(sequence))]
    
    suffixes.sort()     #sortiranje se vrsi po prvoj koordinati!
    
    return suffixes

In [8]:
sequence = 'panamabananas$'

suffix_array = generate_suffix_array(sequence)
print(suffix_array)

[('$', 13), ('abananas$', 5), ('amabananas$', 3), ('anamabananas$', 1), ('ananas$', 7), ('anas$', 9), ('as$', 11), ('bananas$', 6), ('mabananas$', 4), ('namabananas$', 2), ('nanas$', 8), ('nas$', 10), ('panamabananas$', 0), ('s$', 12)]


Funkcija **is_prefix** pokusava da upari sablon **pattern** sa prefiksom niske **suffix**.

In [9]:
def is_prefix(pattern, suffix):
    n = len(pattern)
    m = len(suffix)
    
    if n > m:
        return False
    else:
        return pattern == suffix[:n]

In [10]:
suffix = 'panamabananas'
pattern = 'panama'

print(is_prefix(pattern, sequence))

True


In [11]:
suffix = 'panamabananas'
pattern = 'ana'

print(is_prefix(pattern, sequence))

False


Funkcija **pattern_matching_with_suffix_array** vraca niz ideksa u niski za koju je formiran sufiksni niz **suffix_array** na kojima je pronadjeno uparivanje sa niskom **pattern**.

In [23]:
def pattern_matching_with_suffix_array_bob9952(suffix_array, pattern):
    left_ind = 0
    right_ind = len(suffix_array) - 1

    while left_ind <= right_ind:
        mid_ind = (left_ind + right_ind) // 2
        mid_suffix = suffix_array[mid_ind][0]

        if is_prefix(pattern, mid_suffix):
            # idemo levo i desno dokle god uspevamo da uparimo sablon
            i = mid_ind
            j = mid_ind
            # levo
            while i-1 >= 0:
                if is_prefix(pattern, suffix_array[i-1][0]):
                    i -= 1
                else:
                    break
            # desno
            while j+1 <= right_ind:
                if is_prefix(pattern, suffix_array[j+1][0]):
                    j += 1
                else:
                    break
            # vracamo sufiks u [i, j] delu
            return [elem[1] for elem in suffix_array[i:j+1]] # od i do j 

        # gde sad idemo
        # ako je mid_suffix veci od patterna onda resenje sigurno ne nalazimo u desnom delu, tkd right_ind pomeramo na sredinu i pretragu nastavljamo
        # u delu [levo, sredina]
        if pattern < mid_suffix:
            right_ind = mid_ind
        else:
            left_ind = mid_ind
    return []

In [19]:
def pattern_matching_with_suffix_array(suffix_array, pattern):
    left_ind = 0
    right_ind = len(suffix_array) - 1
    
    while left_ind <= right_ind:
        mid_ind = (left_ind + right_ind) // 2
        mid_suffix = suffix_array[mid_ind][0]
        
        if is_prefix(pattern, mid_suffix):
            #idemo levo i desno dokle god uspevamo da uparimo sablon
            i = mid_ind 
            j = mid_ind 
            
            while i-1 >= 0:
                if is_prefix(pattern, suffix_array[i-1][0]):
                    i -= 1
                else:
                    break
                
            while j+1 < len(suffix_array):
                if is_prefix(pattern, suffix_array[j+1][0]):
                    j += 1
                else:
                    break
            
            return [elem[1] for elem in suffix_array[i:j+1]]
               
        if pattern < mid_suffix:
            right_ind = mid_ind
        else: 
            left_ind = mid_ind
    
    return []

In [20]:
sequence = 'panamabananas$'
pattern = 'ana'

suffix_array = generate_suffix_array(sequence)

print(pattern_matching_with_suffix_array(suffix_array, pattern))

[1, 7, 9]


In [21]:
sequence = 'panamabananas$'
pattern = 'ana'

suffix_array = generate_suffix_array(sequence)

print(pattern_matching_with_suffix_array_bob9952(suffix_array, pattern))

[1, 7, 9]


# Burrows-Wheeler transformacija

U nastavku govorimo o tome na koji nacin mozemo da smanjimo kolicinu memorije potrebne za cuvanje sufiksnog niza (koji za velike niske moze biti poprilicno veliki, npr. citav referentni genom), ali tako da mozemo da je ga rekonstruisemo i da na njega primenimo algoritme za uparivanje sablona.

Burrows-Wheeler transformaciju (krace, BWT) niske dobijamo na sledeci nacin: 
- formirati sve ciklicne rotacije niske (sa dodatim specijalnim terminirajucim karakterom \$)
- dobijene niske sortirati leksikografski ($ ima najmanju leksikografsku vrednost) 
- smestimo sortirane niske u matricu ciklicnih rotacija
- BWT niske je poslednja kolona na ovaj nacin dobijena matrice ciklicnih rotacija 

<img src="assets/BWT.png" width="450"> 

Funkcija **construct_bwt** vraca BW transformaciju niske **sequence**.

In [24]:
def construct_bwt(sequence):
    n = len(sequence)
    
    sequence_cyclic_rotations = [sequence[i:n] + sequence[:i] for i in range(n)]
    sequence_cyclic_rotations.sort()
    
    bwt_sequence = ''.join([rotation[-1] for rotation in sequence_cyclic_rotations])
    
    return bwt_sequence

In [25]:
sequence = 'panamabananas$'

print(construct_bwt(sequence))

smnpbnnaaaaa$a


## Inverzna Burrows-Wheeler transformacija

Rekonstrukciju niske na osnovu njene BWT-niske mozemo da vrsimo na sledeci nacin: 
- sortiranjem BWT-niske, tj. poslednje kolonu matrice ciklicnih rotacija dobijemo prvu kolonu matrice ciklicnih rotacija 
- spojimo poslednju i prvu kolonu, i na taj nacin formiramo 2-gramski sastav ciklicne reprezentacije niske 
- sortiramo 2-gramski sastav i dobijemo prve dve kolone matrice ciklicnih rotacija 
- spojimo poslednju i prethodno rekonstruisane prve dve kolone i na taj nacin formiramo 3-gramski sastav ciklicne reprezentacije niske 
- nastavljamo postupak sve dok ne dobijemo n-gramski sastav, odnosno celu matricu ciklicnih rotacija
- prva ciklicna rotacija bez pocetnog simbola \$ je originalna niska (ili ciklicna rotacija koja u poslednjoj koloni ima simbol \\$)

Funkcija **inverse_bwt** rekonstruise originalnu nisku na osnovu njene BWT-niske **bwt_sequence** tako sto rekonstruise celu matricu ciklicnih rotacija i uzima onu vrstu koja u posledjnjoj koloni ima simbol \$.

In [32]:
import copy

In [33]:
def inverse_bwt(bwt_sequence):
    last_column = list(bwt_sequence)               
    
    columns = copy.deepcopy(last_column)    
    columns.sort()              
    
    for i in range(len(bwt_sequence) - 1):
        for j in range(len(bwt_sequence)):
            columns[j] = last_column[j] + columns[j]
            
        columns.sort()
        
    #print(columns)              #rekonstruisana matrica sortiranih ciklicnih rotacija 
    
    result_index = bwt_sequence.index('$')
    
    return columns[result_index]

In [34]:
sequence = 'panamabananas$'

bwt_sequence = construct_bwt(sequence)

inverse_bwt(bwt_sequence)

'panamabananas$'

# Uparivanje sablona preko BW transformacije niske

Rekonstrukcija niske od njene BWT-niske na prethodno opisan nacin (inverzna BWT) zahteva rekonstrukciju i cuvanje cele matrice ciklicnih rotacija. Originalnu nisku mozemo rekonstruisati i na drugi nacin, bez potrebe za rekonstrukcijom matrice ciklicnih rotacija: 
- sortiranjem BWT-niske, tj. poslednje kolonu matrice ciklicnih rotacija dobijemo prvu kolonu matrice ciklicnih rotacija
- originalnu nisku rekonstruisemo unazad polazeci od simbola \$ u prvoj koloni i trazeci njegovog prethodnika u poslednjoj koloni 
- na taj nacin smo odredili poslednji simbol niske, i u narednom koraku trazimo njegovo pojavljivanje u prvoj koloni da bismo iz njemu odgovarajuce pozicije u poslednjoj koloni odredili pretposlednji simbol originalne niske 
- medjitim, poslednji simbol ne mora imati jedinstveno pojavljivanje u prvoj koloni, pa se postavlja pitanje koji od njih odgovara poslednjem pojavljivanju tog simbola u originalnoj niski <br/>
- **<u>first-last svojstvo</u>: k-to pojavljivanje nekog simbola u prvoj koloni i k-to pojavljivanje tog istog simbola u poslednjoj koloni odgovaraju istoj poziciji simbola u originalnoj niski** 
- na osnovu first-last svojstva pronalazimo odgovarajuce pojavljivanje poslednjeg simbola u prvoj koloni i nastavljamo dalje postupak 

**First-last svojstvo** vazi zato sto ako bismo iz matrice ciklicnih rotacija izdvojili samo one ciklicne rotacije koje pocinju npr. simbolom 'a', i ukoliko bismo svima uklonili taj simbol 'a', dobili bismo niske koje su i dalje sortirane (jer su inicijalno bile leksikografski sortirane). Ako bismo zatim dodali svakoj od njih simbol 'a' na kraj, i dalje ce niske ostati u sortiranom redosledu zato sto smo na sortirane niske dodali svima isti simbol na kraj. Te niske se takodje nalaze u matrici ciklicnih rotacija (jer smo ih dobili tako sto smo ih sve ciklicno zarotirali za 1), i one ce biti u tom ISTOM PORETKU - ne moraju biti uzastopne kao sto su bile pre nego sto smo ih zarotirali ciklicno za 1, ali moraju imati nepromenjen redosled.

Funkcija **last_to_first** na osnovu first-last svojstva pronalazi odgovarajuce pojavljivanje simbola **last_column[index]** u **first_column**.

In [39]:
def last_to_first(last_column, index, first_column):
    character = last_column[index]
    
    last_column_rank = 0           #rang, tj. redni broj pojavljivanja simbola character u poslednjoj koloni
    
    for i in range(index + 1):
        if last_column[i] == character:
            last_column_rank += 1
            
    first_column_rank = 0          #rang, tj. redni broj pojavljivanja simbola character u prvoj koloni
    
    for i in range(len(first_column)):
        if first_column[i] == character:
            first_column_rank += 1 
            
        if first_column_rank == last_column_rank:
            return i

In [40]:
last_column = '$aaaaaabmnnnps'
first_column = 'smnpbnnaaaaa$a'
index = 5

print(last_to_first(last_column, index, first_column))

11


**Sortirana matrica ciklicnih rotacija niske je zapravo isto sto i sufiksni niz niske, s tim sto posle svakog karaktera \$ za sufiksni niz odbacujemo sve sto ide posle.** Zelimo BW transformaciju niske da iskoristimo kao strukturu podataka za 'cuvanje' sufiksnog niza i za uparivanje sablona kao prefiksa nekog od sufiksa.

Skica algoritma: 
- trazimo sva pojavljivanja poslednjeg simbol sablona u BWT-niski tj. poslednjoj koloni matrice ciklicnih rotacija
- na osnovu **first-last svojstva** trazimo odgovarajuca pojavljivanja tih u prvoj koloni matrice ciklicnih rotacija 
- pronalazimo indekse **top** i **bottom** takve da odgovaraju prvom i poslednjem pojavljivanju poslednjeg simbola sablona <u>u prvoj koloni</u>  
- za svaki od indeksa iz opsega **[top, bottom]** posmatramo koji se simbol nalazi u poslednjoj koloni, i medju njima trazimo pretposlednji simbol sablona 
- azuriramo indekse na osnovu **first-last svojstva** tako da vazi da **top** indeks ukazuje na prvo a **bottom** indeks na poslednje pojavljivanje dela sablona koji smo do tada uparili (oba indeksa su u odnosu na prvu kolonu!) 
- ponavljamo isti postupak sve dok ne uparimo sve karaktera sablona
- ukoliko se u nekom od koraka ne pronalazi uparivanje narednog karaktera sablona (tj. prethodnog karaktera posto pretragu vrsimo od pozadi), prekida se pretraga i vraca 0 kao informacija da sablon nije uparen nijednom
- inace, povratna vrednost je broj uparivanja sablona **bottom - top + 1**

Funkcija **bwt_matching** pronalazi na koliko pozicija se niska **pattern** moze upariti sa niskom koja je zadata svojom BWT-niskom **bwt_sequence**.

In [43]:
def bwt_matching(bwt_sequence, pattern):
    first_column = sorted(bwt_sequence)
    last_column = bwt_sequence 
    
    top_ind = 0
    bottom_ind = len(bwt_sequence) - 1
    
    i = len(pattern) - 1    #indeks simbola u pattern-u koji pokusavamo da uparimo
    
    while i >= 0:
        #print('top_ind: {} bottom_ind: {}'.format(top_ind, bottom_ind))
        
        character = pattern[i]
        i -= 1
            
        top_ind_new = -1
        bottom_ind_new = -1
            
        for k in range(top_ind, bottom_ind + 1):
            if top_ind_new == -1 and last_column[k] == character:
                top_ind_new = k
                bottom_ind_new = k
            elif last_column[k] == character:
                bottom_ind_new = k
                
        if top_ind_new == -1 or bottom_ind_new == -1:       #nije pronadjeno uparivanje narednog karaktera
            return 0
            
        top_ind = last_to_first(last_column, top_ind_new, first_column)
        bottom_ind = last_to_first(last_column, bottom_ind_new, first_column)
           
    return bottom_ind - top_ind + 1

In [44]:
sequence = 'panamabananas$'
pattern = 'ana'

bwt_sequence = construct_bwt(sequence)

bwt_matching(bwt_sequence, pattern)

3